In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, load_metric
import random
import numpy as np

In [2]:
# Constants
BATCH_SIZE = 16  # Batch size for training and evaluation
EPOCHS = 3  # Number of training epochs
LEARNING_RATE = 2e-5  # Learning rate for fine-tuning
SEED = 42
TASK_NAME = "sst2"  # GLUE task name

In [3]:
# Set seeds for reproducibility
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    device = torch.device("cuda")
    torch.cuda.empty_cache()  # Clear GPU memory before starting
else:
    device = torch.device("cpu")
    print("Using CPU")

In [4]:
# Load tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("albert-base-v2", trust_remote_code=True)

Loading tokenizer...


In [5]:
# Load GLUE dataset
print(f"Loading GLUE dataset for task: {TASK_NAME}...")
dataset = load_dataset("glue", TASK_NAME)

Loading GLUE dataset for task: sst2...


Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [6]:
# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=128)

In [7]:
print("Tokenizing dataset...")
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Set the format for PyTorch
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Tokenizing dataset...


Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [8]:
# Load pre-trained ALBERT model for sequence classification
print("Loading ALBERT model for sequence classification...")
model = AutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2).to(device)

Loading ALBERT model for sequence classification...


Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
)

In [10]:
# Metrics
accuracy_metric = load_metric("accuracy")
f1_metric = load_metric("f1")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")
    return {**accuracy, **f1}

C:\Users\farha\AppData\Local\Temp\ipykernel_23620\887877820.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy_metric = load_metric("accuracy")
c:\Users\farha\miniconda3\Lib\site-packages\datasets\load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
c:\Users\farha\miniconda3\Lib\site-packages\datasets\load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correct

In [11]:
# Initialize Trainer
print("Initializing Trainer...")
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics,
)

Initializing Trainer...


In [12]:
# Fine-tune the model
print("Starting fine-tuning...")
trainer.train()

Starting fine-tuning...


  0%|          | 0/12630 [00:00<?, ?it/s]

{'loss': 0.6257, 'grad_norm': 28.014461517333984, 'learning_rate': 1.992082343626287e-05, 'epoch': 0.01}
{'loss': 0.434, 'grad_norm': 8.899468421936035, 'learning_rate': 1.9841646872525734e-05, 'epoch': 0.02}
{'loss': 0.3498, 'grad_norm': 133.56211853027344, 'learning_rate': 1.97624703087886e-05, 'epoch': 0.04}
{'loss': 0.4473, 'grad_norm': 116.24050903320312, 'learning_rate': 1.9683293745051467e-05, 'epoch': 0.05}
{'loss': 0.3117, 'grad_norm': 1.8895213603973389, 'learning_rate': 1.9604117181314332e-05, 'epoch': 0.06}
{'loss': 0.3788, 'grad_norm': 102.82685852050781, 'learning_rate': 1.95249406175772e-05, 'epoch': 0.07}
{'loss': 0.3391, 'grad_norm': 34.11256408691406, 'learning_rate': 1.9445764053840065e-05, 'epoch': 0.08}
{'loss': 0.2861, 'grad_norm': 32.15111541748047, 'learning_rate': 1.936658749010293e-05, 'epoch': 0.1}
{'loss': 0.32, 'grad_norm': 34.156578063964844, 'learning_rate': 1.9287410926365797e-05, 'epoch': 0.11}
{'loss': 0.3217, 'grad_norm': 32.44168472290039, 'learning_

  0%|          | 0/55 [00:00<?, ?it/s]

{'eval_loss': 0.24951563775539398, 'eval_accuracy': 0.9208715596330275, 'eval_f1': 0.9208231087370571, 'eval_runtime': 2.6916, 'eval_samples_per_second': 323.967, 'eval_steps_per_second': 20.434, 'epoch': 1.0}
{'loss': 0.1864, 'grad_norm': 46.451168060302734, 'learning_rate': 1.3269992082343628e-05, 'epoch': 1.01}
{'loss': 0.1727, 'grad_norm': 67.98332214355469, 'learning_rate': 1.3190815518606495e-05, 'epoch': 1.02}
{'loss': 0.1381, 'grad_norm': 49.632354736328125, 'learning_rate': 1.311163895486936e-05, 'epoch': 1.03}
{'loss': 0.2621, 'grad_norm': 21.257112503051758, 'learning_rate': 1.3032462391132226e-05, 'epoch': 1.05}
{'loss': 0.2014, 'grad_norm': 51.01828384399414, 'learning_rate': 1.2953285827395092e-05, 'epoch': 1.06}
{'loss': 0.1411, 'grad_norm': 13.032458305358887, 'learning_rate': 1.2874109263657959e-05, 'epoch': 1.07}
{'loss': 0.1927, 'grad_norm': 5.292421817779541, 'learning_rate': 1.2794932699920823e-05, 'epoch': 1.08}
{'loss': 0.187, 'grad_norm': 15.584773063659668, 'le

  0%|          | 0/55 [00:00<?, ?it/s]

{'eval_loss': 0.3881010115146637, 'eval_accuracy': 0.9094036697247706, 'eval_f1': 0.9092812750083504, 'eval_runtime': 2.6133, 'eval_samples_per_second': 333.673, 'eval_steps_per_second': 21.046, 'epoch': 2.0}
{'loss': 0.1238, 'grad_norm': 0.0902155339717865, 'learning_rate': 6.619160728424386e-06, 'epoch': 2.01}
{'loss': 0.1072, 'grad_norm': 154.0924530029297, 'learning_rate': 6.539984164687253e-06, 'epoch': 2.02}
{'loss': 0.1092, 'grad_norm': 0.7110086679458618, 'learning_rate': 6.46080760095012e-06, 'epoch': 2.03}
{'loss': 0.1427, 'grad_norm': 21.517030715942383, 'learning_rate': 6.3816310372129854e-06, 'epoch': 2.04}
{'loss': 0.0972, 'grad_norm': 0.18759001791477203, 'learning_rate': 6.302454473475852e-06, 'epoch': 2.05}
{'loss': 0.0779, 'grad_norm': 0.16151940822601318, 'learning_rate': 6.223277909738717e-06, 'epoch': 2.07}
{'loss': 0.111, 'grad_norm': 26.44621467590332, 'learning_rate': 6.1441013460015846e-06, 'epoch': 2.08}
{'loss': 0.1444, 'grad_norm': 1.1220194101333618, 'learn

  0%|          | 0/55 [00:00<?, ?it/s]

{'eval_loss': 0.40363919734954834, 'eval_accuracy': 0.9174311926605505, 'eval_f1': 0.9174103279012834, 'eval_runtime': 2.6268, 'eval_samples_per_second': 331.959, 'eval_steps_per_second': 20.938, 'epoch': 3.0}
{'train_runtime': 1681.498, 'train_samples_per_second': 120.159, 'train_steps_per_second': 7.511, 'train_loss': 0.1836383237785889, 'epoch': 3.0}


TrainOutput(global_step=12630, training_loss=0.1836383237785889, metrics={'train_runtime': 1681.498, 'train_samples_per_second': 120.159, 'train_steps_per_second': 7.511, 'total_flos': 1207134941575680.0, 'train_loss': 0.1836383237785889, 'epoch': 3.0})

In [13]:
# Evaluate the model on the validation set
print("Evaluating model on validation set...")
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

Evaluating model on validation set...


  0%|          | 0/55 [00:00<?, ?it/s]

Evaluation results: {'eval_loss': 0.24951563775539398, 'eval_accuracy': 0.9208715596330275, 'eval_f1': 0.9208231087370571, 'eval_runtime': 2.6088, 'eval_samples_per_second': 334.255, 'eval_steps_per_second': 21.083, 'epoch': 3.0}


In [14]:
# Save the final model
print("Saving the fine-tuned model...")
model.save_pretrained("./fine_tuned_albert")
tokenizer.save_pretrained("./fine_tuned_albert")
print("Fine-tuning complete and model saved!")

Saving the fine-tuned model...
Fine-tuning complete and model saved!


In [18]:
# Save results in a csv file
import pandas as pd

# Create a DataFrame with the task name and evaluation results
results = pd.DataFrame({"task": [TASK_NAME], **eval_results})

# Save the results to a CSV file
results.to_csv("./results.csv", index=False)